In [1]:
# Initial imports
import pandas as pd
from path import Path
from sklearn import tree
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

import psycopg2
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import session
from sqlalchemy import create_engine, func
import matplotlib.pyplot as plt

In [2]:
#connecting to SQL DB
host_address = 'bootcamp-final-project.c8u2worjd1ui.us-east-1.rds.amazonaws.com'
port = '5432'
username = 'peter_jennifer'
password = 'Puhj6k2%pbW'
db = 'us_gun_violence'


In [3]:
# A long string that contains the necessary Postgres login information
postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'.format(
                username=username,
                password=password,
                ipaddress=host_address,
                port=port,
                dbname=db))
# Create the connection
engine = create_engine(postgres_str)

In [4]:
#make DBs
suspects_df = pd.read_sql_query('''SELECT * FROM suspects;''', engine)
incidents_df = pd.read_sql_query('''SELECT * FROM incidents;''', engine)
guns_df = pd.read_sql_query('''SELECT * FROM guns;''', engine)

guns_df_dup = guns_df.drop_duplicates(subset='incident_id')
guns_df_dup

,gun_index,incident_id,n_guns_involved,gun_stolen,gun_type
0,0,478855,2,Unknown,Unknown
2,2,478959,2,Unknown,Handgun
4,4,479363,2,Unknown,22 LR
6,6,491674,1,Unknown,Unknown
7,7,479413,1,Unknown,Unknown
...,...,...,...,...,...
192446,192446,1083142,1,Unknown,Unknown
192447,192447,1083139,1,Unknown,Unknown
192448,192448,1083151,1,Unknown,Unknown
192449,192449,1082514,1,Unknown,Unknown
